In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-feb-2022/train.csv
/kaggle/input/tabular-playground-series-feb-2022/test.csv


## Get Data

In [2]:
import numpy as np 
import pandas as pd 
import os

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=1.0)
sns.set(style='darkgrid')

import missingno as msno
import time

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer

In [3]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/test.csv")

## EDA

In [4]:
train.shape

(200000, 288)

In [5]:
train.head()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
1,1,-9.536743e-07,-0.000010,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Salmonella_enterica
2,2,-9.536743e-07,-0.000002,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,...,0.000084,0.000048,0.000081,0.000106,0.000072,0.000010,0.000008,0.000019,1.046326e-06,Salmonella_enterica
3,3,4.632568e-08,-0.000006,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,...,0.000151,0.000100,0.000180,0.000202,0.000153,0.000021,0.000015,0.000046,-9.536743e-07,Salmonella_enterica
4,4,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Enterococcus_hirae


In [6]:
train.target.unique()

array(['Streptococcus_pyogenes', 'Salmonella_enterica',
       'Enterococcus_hirae', 'Escherichia_coli', 'Campylobacter_jejuni',
       'Streptococcus_pneumoniae', 'Staphylococcus_aureus',
       'Escherichia_fergusonii', 'Bacteroides_fragilis',
       'Klebsiella_pneumoniae'], dtype=object)

In [7]:
# Number of missing values in each column of training data
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [8]:
train = train.drop("row_id", axis=1)
train.head()

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
1,-9.536743e-07,-0.000010,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Salmonella_enterica
2,-9.536743e-07,-0.000002,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,-0.000007,...,0.000084,0.000048,0.000081,0.000106,0.000072,0.000010,0.000008,0.000019,1.046326e-06,Salmonella_enterica
3,4.632568e-08,-0.000006,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,-0.000007,...,0.000151,0.000100,0.000180,0.000202,0.000153,0.000021,0.000015,0.000046,-9.536743e-07,Salmonella_enterica
4,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Enterococcus_hirae


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 287 entries, A0T0G0C10 to target
dtypes: float64(286), object(1)
memory usage: 437.9+ MB


In [10]:
test.shape

(100000, 287)

In [11]:
test.head()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,200000,-9.536743e-07,-0.000002,-9.153442e-07,0.000024,0.000034,-0.000002,0.000021,0.000024,-0.000009,...,0.000039,0.000085,0.000055,0.000108,0.000090,0.000059,0.000010,0.000006,0.000027,4.632568e-08
1,200001,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,0.001800,-0.000240,0.001800,-0.000114,0.000957,...,-0.000043,0.000914,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07
2,200002,4.632568e-08,0.000003,8.465576e-08,-0.000014,0.000007,-0.000005,-0.000004,0.000003,0.000004,...,0.000041,0.000102,0.000084,0.000111,0.000117,0.000070,0.000020,0.000030,0.000021,4.632568e-08
3,200003,-9.536743e-07,-0.000008,8.084656e-06,0.000216,0.000420,0.000514,0.000452,0.000187,-0.000005,...,0.000069,0.000158,0.000098,0.000175,0.000217,0.000150,0.000018,0.000016,0.000051,-9.536743e-07
4,200004,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,0.000914,-0.000043,-0.000010,-0.000010,0.000990,-9.536743e-07


In [12]:
test = test.drop("row_id", axis=1)
test.head()

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,-9.536743e-07,-0.000002,-9.153442e-07,0.000024,0.000034,-0.000002,0.000021,0.000024,-0.000009,-0.000008,...,0.000039,0.000085,0.000055,0.000108,0.000090,0.000059,0.000010,0.000006,0.000027,4.632568e-08
1,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,0.001800,-0.000240,0.001800,-0.000114,0.000957,-0.000010,...,-0.000043,0.000914,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07
2,4.632568e-08,0.000003,8.465576e-08,-0.000014,0.000007,-0.000005,-0.000004,0.000003,0.000004,-0.000008,...,0.000041,0.000102,0.000084,0.000111,0.000117,0.000070,0.000020,0.000030,0.000021,4.632568e-08
3,-9.536743e-07,-0.000008,8.084656e-06,0.000216,0.000420,0.000514,0.000452,0.000187,-0.000005,-0.000008,...,0.000069,0.000158,0.000098,0.000175,0.000217,0.000150,0.000018,0.000016,0.000051,-9.536743e-07
4,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000043,-0.000086,-0.000043,-0.000086,0.000914,-0.000043,-0.000010,-0.000010,0.000990,-9.536743e-07


In [13]:
convertToNums = {"target": {'Streptococcus_pyogenes':1, 'Salmonella_enterica':2,
                           'Enterococcus_hirae':3, 'Escherichia_coli':4, 
                           'Campylobacter_jejuni':5, 'Streptococcus_pneumoniae':6, 
                           'Staphylococcus_aureus':7,'Escherichia_fergusonii':8, 
                           'Bacteroides_fragilis':9,'Klebsiella_pneumoniae':10}
               }

convertToStrs = {"target": {1:'Streptococcus_pyogenes', 2:'Salmonella_enterica',
                           3:'Enterococcus_hirae', 4:'Escherichia_coli', 
                           5:'Campylobacter_jejuni', 6:'Streptococcus_pneumoniae', 
                           7:'Staphylococcus_aureus',8:'Escherichia_fergusonii', 
                           9:'Bacteroides_fragilis',10:'Klebsiella_pneumoniae'}
               }

In [14]:
train = train.replace(convertToNums)
train.head()

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,1
1,-9.536743e-07,-0.000010,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,2
2,-9.536743e-07,-0.000002,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,-0.000007,...,0.000084,0.000048,0.000081,0.000106,0.000072,0.000010,0.000008,0.000019,1.046326e-06,2
3,4.632568e-08,-0.000006,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,-0.000007,...,0.000151,0.000100,0.000180,0.000202,0.000153,0.000021,0.000015,0.000046,-9.536743e-07,2
4,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,3


## Model - decision tree

In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train = train
X_train = X_train.drop("target", axis=1)
y_train = train['target']
X_test = test

# cross validation
train_X, val_X, train_y, val_y = train_test_split(X_train, y_train, test_size=0.3, random_state = 1)
decision_model = DecisionTreeRegressor()

start_time = time.process_time()
decision_model.fit(train_X, train_y)
print(time.process_time() - start_time, " seconds")

decision_predict = decision_model.predict(val_X)

print("MAE:", mean_absolute_error(val_y, decision_predict))
print("Accuracy:",metrics.accuracy_score(val_y, decision_predict))

# 0.1071, random_state=0
# 0.12005, test_size=0.3, random_state = 1


29.230407715  seconds
MAE: 0.12371666666666667
Accuracy: 0.9655666666666667


## Model - random forest

In [16]:
# from sklearn.ensemble import RandomForestRegressor

# forest_model = RandomForestRegressor(random_state=1)

# start_time = time.process_time()
# forest_model.fit(train_X, train_y)
# print(time.process_time() - start_time, " seconds")

# forest_predict = forest_model.predict(val_X)
# print("MAE:", mean_absolute_error(val_y, forest_predict))

# not good --> 0.1860268

## Model - naive bayes classifier

In [17]:
# from sklearn.naive_bayes import GaussianNB
# nb_model = GaussianNB()

# start_time = time.process_time()
# nb_model.fit(train_X, train_y)
# print(time.process_time() - start_time, " seconds")

# nb_predict = nb_model.predict(val_X) 

# print("MAE:", mean_absolute_error(val_y, nb_predict))
# print("Accuracy:",metrics.accuracy_score(val_y, nb_predict))

# not good --> 2.174

## Model - knn

In [18]:
# from sklearn.neighbors import KNeighborsClassifier
# knn_model = KNeighborsClassifier(n_neighbors=5)

# start_time = time.process_time()
# knn_model.fit(train_X, train_y)
# print(time.process_time() - start_time, " seconds")
# knn_predict = knn_model.predict(val_X) 

# print("MAE:", mean_absolute_error(val_y, knn_predict))
# print("Accuracy:",metrics.accuracy_score(val_y, knn_predict))

# # not good --> 0.179

In [19]:
# from sklearn.metrics import classification_report, confusion_matrix

# start_time = time.process_time()

# print(confusion_matrix(val_y, knn_predict))
# print(classification_report(val_y, knn_predict))

# print(time.process_time() - start_time, " seconds")

In [20]:
# error = []

# start_time = time.process_time()

# # Calculating error for K values between 1 and 40
# for i in range(1, 10):
#     knn = KNeighborsClassifier(n_neighbors=i)
#     knn.fit(train_X, train_y)
#     pred_i = knn.predict(val_X)
#     error.append(np.mean(pred_i != val_y))
    
# print(time.process_time() - start_time, " seconds")  

In [21]:
# plt.figure(figsize=(12, 6))
# plt.plot(range(1, 40), error, color='red', linestyle='dashed', marker='o',
#          markerfacecolor='blue', markersize=10)
# plt.title('Error Rate K Value')
# plt.xlabel('K Value')
# plt.ylabel('Mean Error')

## Model - Artificial neural networks

In [22]:
# mlp for combined regression and classification predictions on the abalone dataset
from numpy import unique
from numpy import argmax
from pandas import read_csv
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model
 
train = train.replace(convertToStrs)    
X_train = train
X_train = X_train.drop("target", axis=1)
y_train = train['target']

n_features = X_train.shape[1]

# encode strings to integer
y_class = LabelEncoder().fit_transform(y_train)
n_class = len(unique(y_class))

# split data into train and test sets
train_X, val_X, y_train, val_y, train_y_class, val_y_class = train_test_split(X_train, y_train, y_class, test_size=0.3, random_state=1)

# input
visible = Input(shape=(n_features,))
hidden1 = Dense(20, activation='relu', kernel_initializer='he_normal')(visible)
hidden2 = Dense(10, activation='relu', kernel_initializer='he_normal')(hidden1)

# regression output
out_reg = Dense(1, activation='linear')(hidden2)
# classification output
out_clas = Dense(n_class, activation='softmax')(hidden2)
# define model
nn_model = Model(inputs=visible, outputs=[out_reg, out_clas])
# compile the keras model
nn_model.compile(loss=['mse','sparse_categorical_crossentropy'], optimizer='adam')
# plot graph of model
plot_model(nn_model, to_file='model.png', show_shapes=True)

# fit the keras model on the dataset
nn_model.fit(train_X, [train_y,train_y_class], epochs=200, batch_size=32, verbose=2)

# make predictions on test set
yhat1, yhat2 = nn_model.predict(val_X)
val_y = val_y.replace(['Streptococcus_pyogenes', 'Salmonella_enterica', 'Enterococcus_hirae', 'Escherichia_coli',
              'Campylobacter_jejuni','Streptococcus_pneumoniae','Staphylococcus_aureus','Escherichia_fergusonii',
              'Bacteroides_fragilis','Klebsiella_pneumoniae'], 
              [1,2,3,4,5,6,7,8,9,10])

# calculate error for regression model
error = mean_absolute_error(val_y, yhat1)
print('MAE: %.3f' % error)

# evaluate accuracy for classification model
yhat2 = argmax(yhat2, axis=-1).astype('int')
acc = accuracy_score(val_y_class, yhat2)
print('Accuracy: %.3f' % acc)

2022-02-27 07:26:29.396404: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-27 07:26:29.480205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-27 07:26:29.480950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-27 07:26:29.482112: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/200
4375/4375 - 9s - loss: 9.6017 - dense_2_loss: 7.7348 - dense_3_loss: 1.8668
Epoch 2/200
4375/4375 - 8s - loss: 6.8403 - dense_2_loss: 5.4947 - dense_3_loss: 1.3456
Epoch 3/200
4375/4375 - 7s - loss: 5.9023 - dense_2_loss: 4.6894 - dense_3_loss: 1.2129
Epoch 4/200
4375/4375 - 7s - loss: 5.3107 - dense_2_loss: 4.1929 - dense_3_loss: 1.1179
Epoch 5/200
4375/4375 - 7s - loss: 4.7786 - dense_2_loss: 3.7451 - dense_3_loss: 1.0335
Epoch 6/200
4375/4375 - 8s - loss: 4.3189 - dense_2_loss: 3.3556 - dense_3_loss: 0.9633
Epoch 7/200
4375/4375 - 7s - loss: 3.9930 - dense_2_loss: 3.0865 - dense_3_loss: 0.9065
Epoch 8/200
4375/4375 - 7s - loss: 3.7465 - dense_2_loss: 2.8816 - dense_3_loss: 0.8649
Epoch 9/200
4375/4375 - 7s - loss: 3.5326 - dense_2_loss: 2.7068 - dense_3_loss: 0.8258
Epoch 10/200
4375/4375 - 7s - loss: 3.3572 - dense_2_loss: 2.5647 - dense_3_loss: 0.7925
Epoch 11/200
4375/4375 - 8s - loss: 3.2098 - dense_2_loss: 2.4462 - dense_3_loss: 0.7635
Epoch 12/200
4375/4375 - 7s - 

In [23]:
nn_predict = nn_model.predict(X_test)
predict = nn_predict[0]

## Model - LightGBM

In [24]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier()

start_time = time.process_time()
lgb_model.fit(train_X, train_y)
print(time.process_time() - start_time, " seconds")

lgb_predict = lgb_model.predict(val_X) 

print("MAE:", mean_absolute_error(val_y, lgb_predict))
print("Accuracy:",metrics.accuracy_score(val_y, lgb_predict))

345.375937322  seconds
MAE: 0.07753333333333333
Accuracy: 0.9792333333333333


## Submit

In [25]:
## select model
predict = lgb_model.predict(X_test)

In [26]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv")
submission.head()

,row_id,target
0,200000,Streptococcus_pneumoniae
1,200001,Streptococcus_pneumoniae
2,200002,Streptococcus_pneumoniae
3,200003,Streptococcus_pneumoniae
4,200004,Streptococcus_pneumoniae


In [27]:
predict

array([8, 2, 3, ..., 9, 9, 1])

In [28]:

#submission["target"] = np.round(predict[0])
submission["target"] = predict
submission = submission.replace(convertToStrs)

submission.head()

,row_id,target
0,200000,Escherichia_fergusonii
1,200001,Salmonella_enterica
2,200002,Enterococcus_hirae
3,200003,Salmonella_enterica
4,200004,Staphylococcus_aureus


In [29]:
submission.to_csv("./submission_lgbm.csv", index=False)